---

## 1. Učitavanje potrebnih biblioteka i učitavanje podataka

Prvo ćemo učitati sve potrebne biblioteke za analizu podataka:

In [2]:
# Učitavanje potrebnih biblioteka
import pandas as pd         
import numpy as np           

# Prikaži verzije biblioteka
print("Verzije biblioteka:")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

# Učitavanje podataka iz CSV datoteke iz dataset mape
# read_csv je standardna Pandas funkcija za učitavanje podataka iz CSV formata
df_original = pd.read_csv('../dataset/student-por.csv', sep=';')

# Kreiramo radnu kopiju s kojom ćemo raditi (backup originala u memoriji)
df = df_original.copy()

print("Datoteka uspješno učitana!")
print(f"\nDimenzije datoteke: {df.shape}")
print(f"  - Broj redaka (zapisa): {df.shape[0]}")
print(f"  - Broj stupaca (atributa): {df.shape[1]}")

Verzije biblioteka:
Pandas: 2.3.3
NumPy: 2.4.0
Datoteka uspješno učitana!

Dimenzije datoteke: (649, 33)
  - Broj redaka (zapisa): 649
  - Broj stupaca (atributa): 33


### Prikaz Osnovnih Informacija o DataFrameu

**Što prikazujemo:**
- Nazive svih stupaca (atributa)
- Tipove podataka u svakom stupcu
- Broj ne-nedostajućih vrijednosti
- Memorijsku potrošnju

In [3]:
# Detaljne informacije o DataFrameu
print("INFORMACIJE O DATAFREMU - PRIJE OBRADE:\n")
df.info()

INFORMACIJE O DATAFREMU - PRIJE OBRADE:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      649 non-null    object
 1   sex         649 non-null    object
 2   age         649 non-null    int64 
 3   address     649 non-null    object
 4   famsize     649 non-null    object
 5   Pstatus     649 non-null    object
 6   Medu        649 non-null    int64 
 7   Fedu        649 non-null    int64 
 8   Mjob        649 non-null    object
 9   Fjob        649 non-null    object
 10  reason      649 non-null    object
 11  guardian    649 non-null    object
 12  traveltime  649 non-null    int64 
 13  studytime   649 non-null    int64 
 14  failures    649 non-null    int64 
 15  schoolsup   649 non-null    object
 16  famsup      649 non-null    object
 17  paid        649 non-null    object
 18  activities  649 non-null    object
 19  nursery  

### Prikaz Prvih Redaka

Pogledajmo kako izgledaju prvi redci dataseta:

In [4]:
# Prikazi prvih 10 redaka
print("PRVIH 10 REDAKA DATASETA:\n")
print(df.head(10))
print("\n" + "="*60 + "\n")

PRVIH 10 REDAKA DATASETA:

  school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4   at_home   teacher   
1     GP   F   17       U     GT3       T     1     1   at_home     other   
2     GP   F   15       U     LE3       T     1     1   at_home     other   
3     GP   F   15       U     GT3       T     4     2    health  services   
4     GP   F   16       U     GT3       T     3     3     other     other   
5     GP   M   16       U     LE3       T     4     3  services     other   
6     GP   M   16       U     LE3       T     2     2     other     other   
7     GP   F   17       U     GT3       A     4     4     other   teacher   
8     GP   M   15       U     LE3       A     3     2  services     other   
9     GP   M   15       U     GT3       T     3     4     other     other   

   ... famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0  ...      4        3      4     1     1      3     

---

## 2. Otkrivanje Nedostajućih Vrijednosti

Sada ćemo analizirati gdje su nedostajuće vrijednosti u našem datasetu. Pandas koristi `NaN` (Not a Number) za označavanje nedostajućih vrijednosti.


### Prikaz redaka s nedostajućim vrijednostima

`.isnull().any(axis=1)` provjerava je li bilo koja vrijednost u redu NaN (`any(axis=1)` znači "provjeravaj redove" - axis=1), što nam daje sve redake koji imaju barem jednu nedostajuću vrijednost.

In [5]:
# Pronađi sve redove koji imaju BAR JEDNU nedostajuću vrijednost
# .any(axis=1) provjerava po redcima (axis=1) - vraća True ako red ima bar jednu NaN vrijednost
rows_with_missing = df[df.isnull().any(axis=1)]

print(f"REDCI S NEDOSTAJUĆIM VRIJEDNOSTIMA ({len(rows_with_missing)} redaka):\n")
print(rows_with_missing)

REDCI S NEDOSTAJUĆIM VRIJEDNOSTIMA (0 redaka):

Empty DataFrame
Columns: [school, sex, age, address, famsize, Pstatus, Medu, Fedu, Mjob, Fjob, reason, guardian, traveltime, studytime, failures, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic, famrel, freetime, goout, Dalc, Walc, health, absences, G1, G2, G3]
Index: []

[0 rows x 33 columns]


---

## 3. Rekodiranje Nedostajućih Vrijednosti

Sada ćemo **zamijeniti sve nedostajuće vrijednosti (NaN)** sa stringom **"missing"**.

### Zašto rekodirati nedostajuće vrijednosti?

1. **Jasnoća** - lakše je razumjeti podatke kada je eksplicitno navedeno što je "nedostaje"
2. **Analiza** - "missing" je stringovna vrijednost koja se može analizirati kao kategorija
3. **Usporedba** - lakše se može pregledati koji stupac ima više nedostajućih vrijednosti
4. **Čuvanje podataka** - čuva se informacija da je vrijednost nedostajala, za razliku od brisanja redaka

### Metoda: fillna() - Zamjena nedostajućih vrijednosti

`.fillna('missing')` zamjenjuje sve NaN vrijednosti navedenom vrijednosti (u ovom slučaju stringom "missing"), što omogućava da nedostajuće vrijednosti postanu vidljive kao eksplicitne kategorije umjesto numeričkog NaN.

In [6]:
# Zamjena svih NaN vrijednosti sa stringom "missing"
# fillna() metoda zamjenjuje sve NaN vrijednosti navedenom vrijednosti

print("REKODIRANJE NEDOSTAJUĆIH VRIJEDNOSTI:\n")
print("Korištenje .fillna('missing')...\n")

df_cleaned = df.fillna('missing')

print("Rekodiranje uspješno. Sve NaN vrijednosti zamijenjene sa 'missing'.")
print("\n" + "="*60 + "\n")

REKODIRANJE NEDOSTAJUĆIH VRIJEDNOSTI:

Korištenje .fillna('missing')...

Rekodiranje uspješno. Sve NaN vrijednosti zamijenjene sa 'missing'.




### Provjera rezultata rekodiranja

Pogledajmo redake s nedostajućim vrijednostima nakon rekodiranja:

In [7]:
# Prikaži redake koji su prije imali nedostajuće vrijednosti
print("REDCI NAKON REKODIRANJA (koji su prije imali nedostajuće vrijednosti):\n")
print(rows_with_missing.index.tolist())
print("\nPrikaz tih redaka iz očišćenog dataseta:\n")
print(df_cleaned.loc[rows_with_missing.index])
print("\n" + "="*60 + "\n")

REDCI NAKON REKODIRANJA (koji su prije imali nedostajuće vrijednosti):

[]

Prikaz tih redaka iz očišćenog dataseta:

Empty DataFrame
Columns: [school, sex, age, address, famsize, Pstatus, Medu, Fedu, Mjob, Fjob, reason, guardian, traveltime, studytime, failures, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic, famrel, freetime, goout, Dalc, Walc, health, absences, G1, G2, G3]
Index: []

[0 rows x 33 columns]




---

## 4. Inicijalno Sondiranje Podataka - Identifikacija Neupotrebljivih Atributa

Cilj ovog koraka je utvrditi koji atributi **nisu upotrebljivi** za analitiku zbog:
- **Nedostatka varijacije** (uniformne vrijednosti)
- **Logički nemogućih vrijednosti** (pogreške pri unosu)
- **Prevelikog postotka nedostajućih vrijednosti**

### 4.1 Identifikacija Uniformnosti pomoću Standardne Devijacije

Standardna devijacija mjeri raspršenost podataka oko srednje vrijednosti. Ako je standardna devijacija **0 ili blizu 0**, to znači da svi podaci imaju istu ili gotovo istu vrijednost - takvi atributi ne pružaju **informacijsku vrijednost** za modeliranje jer nema varijacije koju bi model mogao naučiti.


In [8]:
# Izračunavanje standardne devijacije za sve numeričke stupce
# .select_dtypes() filtrira samo numeričke tipove podataka (int64, float64)
# .std() računa standardnu devijaciju za svaki stupac

print("ANALIZA UNIFORMNOSTI - STANDARDNA DEVIJACIJA NUMERIČKIH ATRIBUTA:\n")

# Izdvajamo samo numeričke stupce
numeric_cols = df.select_dtypes(include=[np.number])
print(f"Broj numeričkih stupaca: {len(numeric_cols.columns)}\n")

# Računamo standardnu devijaciju
std_values = numeric_cols.std().sort_values()

print("Standardna devijacija po stupcima (sortirano uzlazno):\n")
print(std_values.round(4))

# Identificiramo stupce s niskom varijacijom (std < 0.5)
low_variance = std_values[std_values < 0.5]
print(f"\n{'='*60}")
print(f"\nSTUPCI S NISKOM VARIJACIJOM (std < 0.5):")
if len(low_variance) > 0:
    for col, std in low_variance.items():
        print(f"  - {col}: std = {std:.4f}")
else:
    print("  Nema stupaca s uniformnim vrijednostima.")

# Stupci sa std = 0 (potpuno uniformni)
zero_variance = std_values[std_values == 0]
print(f"\nSTUPCI S NULTOM VARIJACIJOM (std = 0) - KANDIDATI ZA ELIMINACIJU:")
if len(zero_variance) > 0:
    for col in zero_variance.index:
        print(f"  - {col} (svi zapisi imaju istu vrijednost)")
else:
    print("  Nema potpuno uniformnih stupaca - svi atributi imaju varijaciju.")


ANALIZA UNIFORMNOSTI - STANDARDNA DEVIJACIJA NUMERIČKIH ATRIBUTA:

Broj numeričkih stupaca: 16

Standardna devijacija po stupcima (sortirano uzlazno):

failures      0.5932
traveltime    0.7487
studytime     0.8295
Dalc          0.9248
famrel        0.9557
freetime      1.0511
Fedu          1.0999
Medu          1.1346
goout         1.1758
age           1.2181
Walc          1.2844
health        1.4463
G1            2.7453
G2            2.9136
G3            3.2307
absences      4.6408
dtype: float64


STUPCI S NISKOM VARIJACIJOM (std < 0.5):
  Nema stupaca s uniformnim vrijednostima.

STUPCI S NULTOM VARIJACIJOM (std = 0) - KANDIDATI ZA ELIMINACIJU:
  Nema potpuno uniformnih stupaca - svi atributi imaju varijaciju.


### 4.2 Provjera Logičkih Granica (Min/Max Analiza)

Uspoređujemo stvarne minimalne i maksimalne vrijednosti u podacima s **dokumentiranim granicama** iz `student.txt`. Ova provjera otkriva:
- **Pogreške pri unosu podataka** (npr. dob = 100)
- **Nelogične vrijednosti** (npr. ocjena G1 > 20 kada je maksimum 20)
- **Outliere** koji mogu utjecati na kvalitetu analize

**Očekivane granice prema dokumentaciji:**
| Atribut | Min | Max | Opis |
|---------|-----|-----|------|
| age | 15 | 22 | Dob učenika |
| Medu/Fedu | 0 | 4 | Obrazovanje roditelja |
| traveltime | 1 | 4 | Vrijeme putovanja |
| studytime | 1 | 4 | Vrijeme učenja tjedno |
| failures | 0 | 4 | Broj padova |
| famrel/freetime/goout/Dalc/Walc/health | 1 | 5 | Likertova skala |
| absences | 0 | 93 | Broj izostanaka |
| G1/G2/G3 | 0 | 20 | Ocjene |


In [ ]:
# Provjera logičkih granica - usporedba min/max vrijednosti s dokumentacijom
# Definiramo očekivane granice prema student.txt dokumentaciji

print("PROVJERA LOGIČKIH GRANICA NUMERIČKIH ATRIBUTA:\n")

# Definirajmo očekivane granice prema dokumentaciji
expected_bounds = {
    'age': (15, 22),
    'Medu': (0, 4),
    'Fedu': (0, 4),
    'traveltime': (1, 4),
    'studytime': (1, 4),
    'failures': (0, 4),
    'famrel': (1, 5),
    'freetime': (1, 5),
    'goout': (1, 5),
    'Dalc': (1, 5),
    'Walc': (1, 5),
    'health': (1, 5),
    'absences': (0, 93),
    'G1': (0, 20),
    'G2': (0, 20),
    'G3': (0, 20)
}

# Izračunaj stvarne min i max vrijednosti
actual_min = numeric_cols.min()
actual_max = numeric_cols.max()

# Kreiraj DataFrame za pregled
bounds_check = pd.DataFrame({
    'Stvarni_Min': actual_min,
    'Stvarni_Max': actual_max,
    'Očekivani_Min': [expected_bounds.get(col, (None, None))[0] for col in numeric_cols.columns],
    'Očekivani_Max': [expected_bounds.get(col, (None, None))[1] for col in numeric_cols.columns]
})

print(bounds_check)
print(f"\n{'='*60}")

# Provjeri ima li vrijednosti izvan očekivanih granica
print("\nPROVJERA NELOGIČNIH VRIJEDNOSTI:\n")
issues_found = False

for col in numeric_cols.columns:
    if col in expected_bounds:
        exp_min, exp_max = expected_bounds[col]
        act_min, act_max = actual_min[col], actual_max[col]
        
        if act_min < exp_min:
            print(f"⚠️  {col}: Stvarni minimum ({act_min}) je MANJI od očekivanog ({exp_min})")
            issues_found = True
        if act_max > exp_max:
            print(f"⚠️  {col}: Stvarni maksimum ({act_max}) je VEĆI od očekivanog ({exp_max})")
            issues_found = True

if not issues_found:
    print("✓ Sve vrijednosti su unutar očekivanih logičkih granica.")
    print("  Nema nelogičnih vrijednosti koje bi ukazivale na pogreške pri unosu.")


PROVJERA LOGIČKIH GRANICA NUMERIČKIH ATRIBUTA:

            Stvarni_Min  Stvarni_Max  Očekivani_Min  Očekivani_Max
age                  15           22             15             22
Medu                  0            4              0              4
Fedu                  0            4              0              4
traveltime            1            4              1              4
studytime             1            4              1              4
failures              0            3              0              4
famrel                1            5              1              5
freetime              1            5              1              5
goout                 1            5              1              5
Dalc                  1            5              1              5
Walc                  1            5              1              5
health                1            5              1              5
absences              0           32              0             93
G1            

### 4.3 Provjera Popunjenosti - Analiza Nedostajućih Vrijednosti

Izračunavamo **postotak nedostajućih vrijednosti** po svakom atributu. Prema metodološkim preporukama:
- Atributi s **više od 20% nedostajućih** vrijednosti mogu ozbiljno ugroziti kvalitetu analize
- Takvi atributi su kandidati za **eliminaciju** ili zahtijevaju posebne tehnike imputacije

Ova provjera nadopunjuje prethodnu analizu iz koraka 2 (rekodiranje) s kvantitativnom perspektivom.


In [ ]:
# Analiza nedostajućih vrijednosti - postotak po atributu
# Koristimo originalni DataFrame (df) jer df_cleaned već ima zamijenjene NaN vrijednosti

print("ANALIZA POPUNJENOSTI - NEDOSTAJUĆE VRIJEDNOSTI PO ATRIBUTU:\n")

# Izračunaj broj nedostajućih vrijednosti po stupcu
missing_count = df.isnull().sum()

# Izračunaj postotak nedostajućih vrijednosti
missing_percentage = (missing_count / len(df) * 100).round(2)

# Kreiraj pregledan DataFrame
missing_analysis = pd.DataFrame({
    'Nedostaje_Broj': missing_count,
    'Nedostaje_%': missing_percentage,
    'Popunjeno_%': (100 - missing_percentage).round(2)
}).sort_values('Nedostaje_%', ascending=False)

print(missing_analysis)
print(f"\n{'='*60}")

# Identificiraj atribute s više od 20% nedostajućih vrijednosti
critical_missing = missing_analysis[missing_analysis['Nedostaje_%'] > 20]

print("\nATRIBUTI S VIŠE OD 20% NEDOSTAJUĆIH VRIJEDNOSTI (kritični za analizu):\n")
if len(critical_missing) > 0:
    for col in critical_missing.index:
        pct = critical_missing.loc[col, 'Nedostaje_%']
        print(f"  ⚠️  {col}: {pct}% nedostajućih - KANDIDAT ZA ELIMINACIJU")
else:
    print("  ✓ Nema atributa s kritičnim postotkom nedostajućih vrijednosti.")
    print("  Svi atributi imaju dovoljnu popunjenost za kvalitetnu analizu.")


ANALIZA POPUNJENOSTI - NEDOSTAJUĆE VRIJEDNOSTI PO ATRIBUTU:

            Nedostaje_Broj  Nedostaje_%  Popunjeno_%
school                   0          0.0        100.0
paid                     0          0.0        100.0
G2                       0          0.0        100.0
G1                       0          0.0        100.0
absences                 0          0.0        100.0
health                   0          0.0        100.0
Walc                     0          0.0        100.0
Dalc                     0          0.0        100.0
goout                    0          0.0        100.0
freetime                 0          0.0        100.0
famrel                   0          0.0        100.0
romantic                 0          0.0        100.0
internet                 0          0.0        100.0
higher                   0          0.0        100.0
nursery                  0          0.0        100.0
activities               0          0.0        100.0
famsup                   0          0.

### 4.4 Sažetak - Varijable za Eliminaciju

Na temelju provedene analize, ovdje ćemo sumirati koje varijable treba eliminirati iz daljnje obrade:


In [11]:
# Sažetak analize - popis varijabli za eliminaciju

print("=" * 60)
print("SAŽETAK ANALIZE KVALITETE PODATAKA")
print("=" * 60)

# Lista varijabli za eliminaciju
variables_to_eliminate = []

# 1. Uniformne varijable (std = 0)
if len(zero_variance) > 0:
    variables_to_eliminate.extend(zero_variance.index.tolist())
    print(f"\n1. UNIFORMNE VARIJABLE (std = 0): {list(zero_variance.index)}")
else:
    print("\n1. UNIFORMNE VARIJABLE: Nema")

# 2. Varijable s nelogičnim vrijednostima
illogical_vars = []
for col in numeric_cols.columns:
    if col in expected_bounds:
        exp_min, exp_max = expected_bounds[col]
        if actual_min[col] < exp_min or actual_max[col] > exp_max:
            illogical_vars.append(col)

if illogical_vars:
    variables_to_eliminate.extend(illogical_vars)
    print(f"\n2. NELOGIČNE VRIJEDNOSTI: {illogical_vars}")
else:
    print("\n2. NELOGIČNE VRIJEDNOSTI: Nema")

# 3. Varijable s previše nedostajućih vrijednosti
if len(critical_missing) > 0:
    variables_to_eliminate.extend(critical_missing.index.tolist())
    print(f"\n3. KRITIČNO NEDOSTAJUĆE (>20%): {list(critical_missing.index)}")
else:
    print("\n3. KRITIČNO NEDOSTAJUĆE: Nema")

# Ukloni duplikate
variables_to_eliminate = list(set(variables_to_eliminate))

print(f"\n{'='*60}")
print(f"\nKONAČAN POPIS VARIJABLI ZA ELIMINACIJU: {variables_to_eliminate if variables_to_eliminate else 'Nema'}")
print(f"\nZAKLJUČAK: ", end="")
if variables_to_eliminate:
    print(f"Identificirano je {len(variables_to_eliminate)} varijabli koje treba razmotriti za eliminaciju.")
else:
    print("Svi atributi prolaze kontrolu kvalitete i mogu se koristiti u daljnjoj analizi.")


SAŽETAK ANALIZE KVALITETE PODATAKA

1. UNIFORMNE VARIJABLE: Nema

2. NELOGIČNE VRIJEDNOSTI: Nema

3. KRITIČNO NEDOSTAJUĆE: Nema


KONAČAN POPIS VARIJABLI ZA ELIMINACIJU: Nema

ZAKLJUČAK: Svi atributi prolaze kontrolu kvalitete i mogu se koristiti u daljnjoj analizi.


---

## 5. Određivanje, Provjera i Redizajn Ciljne Varijable

Ciljna varijabla (target) je ono što želimo predvidjeti modelom. U ovom slučaju, transformiramo numeričku ocjenu **G1** (prva periodična ocjena) u **binomnu varijablu** koja označava rizik od pada.

### 5.1 Kreiranje Binomne Ciljne Varijable

Transformacija:
- **target_G1 = 1** → Ako je G1 < 10 (pad/rizik) 
- **target_G1 = 0** → Ako je G1 >= 10 (prolaz)

Ova binarna klasifikacija omogućuje primjenu klasifikacijskih algoritama za predviđanje učenika u riziku.


In [17]:
# Kreiranje binomne ciljne varijable target_G1
# Koristi se numpy where() funkcija za kondicionalno kreiranje vrijednosti
# Sintaksa: np.where(uvjet, vrijednost_ako_true, vrijednost_ako_false)

print("KREIRANJE BINOMNE CILJNE VARIJABLE:\n")

# Prikaži distribuciju originalnog G1 stupca
print("Distribucija originalnog G1 stupca:")
print(f"  - Minimum: {df['G1'].min()}")
print(f"  - Maksimum: {df['G1'].max()}")
print(f"  - Prosjek: {df['G1'].mean():.2f}")
print(f"  - Medijan: {df['G1'].median()}")

# Kreiranje target varijable
# 1 = rizik (G1 < 10), 0 = prolaz (G1 >= 10)
df['target_G1'] = np.where(df['G1'] < 10, 1, 0)

print(f"\n{'='*60}")
print("\nBinomna ciljna varijabla 'target_G1' uspješno kreirana!")
print("  - Vrijednost 1: Učenik u riziku (G1 < 10)")
print("  - Vrijednost 0: Učenik prolazi (G1 >= 10)")


KREIRANJE BINOMNE CILJNE VARIJABLE:

Distribucija originalnog G1 stupca:
  - Minimum: 0
  - Maksimum: 19
  - Prosjek: 11.40
  - Medijan: 11.0


Binomna ciljna varijabla 'target_G1' uspješno kreirana!
  - Vrijednost 1: Učenik u riziku (G1 < 10)
  - Vrijednost 0: Učenik prolazi (G1 >= 10)


### 5.2 Provjera Kvalitete Ciljne Varijable kroz Frekvenciju

Kvalitetna ciljna varijabla mora imati **zastupljena oba stanja** (rizik/prolaz). Ako je jedna klasa previše dominantna (npr. 99% vs 1%), model neće moći naučiti razlikovati klase - to se naziva **problem nebalansiranih klasa**.

Preporučeni omjer: Manjinska klasa bi trebala imati barem **10-15%** zastupljenosti za pouzdano modeliranje.


In [18]:
# Provjera distribucije ciljne varijable
# value_counts() broji frekvenciju svake jedinstvene vrijednosti

print("DISTRIBUCIJA CILJNE VARIJABLE (target_G1):\n")

# Frekvencija - apsolutni brojevi
freq = df['target_G1'].value_counts().sort_index()
print("Apsolutna frekvencija:")
print(f"  - Prolaz (0): {freq[0]} učenika")
print(f"  - Rizik  (1): {freq[1]} učenika")

# Frekvencija - postoci
freq_pct = df['target_G1'].value_counts(normalize=True).sort_index() * 100
print(f"\nRelativna frekvencija (postoci):")
print(f"  - Prolaz (0): {freq_pct[0]:.2f}%")
print(f"  - Rizik  (1): {freq_pct[1]:.2f}%")

# Omjer klasa
ratio = freq[0] / freq[1] if freq[1] > 0 else float('inf')
print(f"\nOmjer klasa (prolaz:rizik): {ratio:.2f}:1")

print(f"\n{'='*60}")

# Procjena kvalitete
min_class_pct = min(freq_pct)
print(f"\nPROCJENA KVALITETE CILJNE VARIJABLE:")
if min_class_pct >= 15:
    print(f"  ✓ Odlična zastupljenost: Manjinska klasa ima {min_class_pct:.2f}%")
    print("    Ciljna varijabla je pogodna za modeliranje.")
elif min_class_pct >= 10:
    print(f"  ⚠️ Prihvatljiva zastupljenost: Manjinska klasa ima {min_class_pct:.2f}%")
    print("    Modeliranje je moguće, ali razmotriti tehnike balansiranja.")
else:
    print(f"  ❌ Nebalansirana: Manjinska klasa ima samo {min_class_pct:.2f}%")
    print("    Preporuča se primjena tehnika balansiranja (SMOTE, undersampling).")


DISTRIBUCIJA CILJNE VARIJABLE (target_G1):

Apsolutna frekvencija:
  - Prolaz (0): 492 učenika
  - Rizik  (1): 157 učenika

Relativna frekvencija (postoci):
  - Prolaz (0): 75.81%
  - Rizik  (1): 24.19%

Omjer klasa (prolaz:rizik): 3.13:1


PROCJENA KVALITETE CILJNE VARIJABLE:
Odlična zastupljenost: Manjinska klasa ima 24.19%
Ciljna varijabla je pogodna za modeliranje.


### 5.3 Vremensko Usklađivanje Prediktora

Ključno pravilo modeliranja: **Prediktori moraju vremenski prethoditi ciljnoj varijabli**. To osigurava:
- **Kauzalnu logiku** - uzrok mora prethoditi posljedici
- **Praktičnu primjenjivost** - model mora koristiti podatke dostupne PRIJE ishoda
- **Izbjegavanje "curenja podataka"** (data leakage) - kada prediktor sadrži informaciju o ishodu

Za G1 (prva periodična ocjena), moramo osigurati da svi prediktori odražavaju stanje PRIJE ocjenjivanja.


In [19]:
# Analiza vremenskog usklađivanja prediktora s ciljnom varijablom G1

print("ANALIZA VREMENSKOG USKLAĐIVANJA PREDIKTORA:\n")

# Kategorizacija atributa prema vremenskom odnosu s G1
# Atributi prikupljeni PRIJE ocjenjivanja (validni prediktori)
predictors_before_g1 = [
    'school', 'sex', 'age', 'address', 'famsize', 'Pstatus',  # Demografija
    'Medu', 'Fedu', 'Mjob', 'Fjob',  # Obiteljska pozadina
    'reason', 'guardian',  # Izbor škole i skrbništvo
    'traveltime', 'studytime', 'failures',  # Akademski faktori prije G1
    'schoolsup', 'famsup', 'paid', 'activities', 'nursery',  # Podrška i aktivnosti
    'higher', 'internet', 'romantic',  # Aspiracije i životni stil
    'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health',  # Psihosocijalni faktori
    'absences'  # Izostanci (pretpostavljamo da se mjere prije G1)
]

# Atributi koji NISU validni prediktori (nastaju nakon ili istovremeno s G1)
invalid_predictors = ['G1', 'G2', 'G3', 'target_G1']

print("VALIDNI PREDIKTORI (prikupljeni prije G1):")
print("-" * 50)
for i, pred in enumerate(predictors_before_g1, 1):
    if pred in df.columns:
        print(f"  {i:2d}. {pred}")

print(f"\nUkupno validnih prediktora: {len(predictors_before_g1)}")

print(f"\n{'='*60}")
print("\nNEVALIDNI PREDIKTORI (data leakage rizik):")
print("-" * 50)
for pred in invalid_predictors:
    if pred in df.columns:
        reason = ""
        if pred == 'G1':
            reason = "← Izvor ciljne varijable"
        elif pred == 'G2':
            reason = "← Nastaje NAKON G1"
        elif pred == 'G3':
            reason = "← Finalna ocjena, nastaje NAKON G1"
        elif pred == 'target_G1':
            reason = "← Ciljna varijabla (ne može biti prediktor)"
        print(f"  ⚠️  {pred} {reason}")

print(f"\n{'='*60}")
print("\nZAKLJUČAK:")
print("  Svi prediktori (osim G1, G2, G3) vremenski prethode ocjenjivanju G1.")
print("  Model može koristiti ove atribute za predviđanje rizika učenika.")


ANALIZA VREMENSKOG USKLAĐIVANJA PREDIKTORA:

VALIDNI PREDIKTORI (prikupljeni prije G1):
--------------------------------------------------
   1. school
   2. sex
   3. age
   4. address
   5. famsize
   6. Pstatus
   7. Medu
   8. Fedu
   9. Mjob
  10. Fjob
  11. reason
  12. guardian
  13. traveltime
  14. studytime
  15. failures
  16. schoolsup
  17. famsup
  18. paid
  19. activities
  20. nursery
  21. higher
  22. internet
  23. romantic
  24. famrel
  25. freetime
  26. goout
  27. Dalc
  28. Walc
  29. health
  30. absences

Ukupno validnih prediktora: 30


NEVALIDNI PREDIKTORI (data leakage rizik):
--------------------------------------------------
 G1 <- Izvor ciljne varijable
 G2 <- Nastaje NAKON G1
 G3 <- Finalna ocjena, nastaje NAKON G1
 target_G1 <- Ciljna varijabla (ne može biti prediktor)


ZAKLJUČAK:
  Svi prediktori (osim G1, G2, G3) vremenski prethode ocjenjivanju G1.
  Model može koristiti ove atribute za predviđanje rizika učenika.


### 5.4 Redizajn Ciljne Varijable (Spiralni Pristup)

Ako preliminarni pregled pokaže probleme s ciljnom varijablom (npr. nema učenika s ocjenom < 10), potrebno je primijeniti **spiralni pristup**:

1. **Prilagodba praga** - npr. umjesto G1 < 10, koristiti G1 < 12
2. **Promjena uzorka** - uključiti dodatne podatke ili koristiti stratificirano uzorkovanje
3. **Promjena ciljne varijable** - umjesto binarizacije, koristiti višeklasnu klasifikaciju

Ovdje provjeravamo je li redizajn potreban i predlažemo alternativne pragove ako je potrebno.


In [21]:
# Analiza potrebe za redizajnom ciljne varijable

print("ANALIZA POTREBE ZA REDIZAJNOM CILJNE VARIJABLE:\n")

# Trenutna distribucija
risk_count = (df['G1'] < 10).sum()
pass_count = (df['G1'] >= 10).sum()
risk_pct = risk_count / len(df) * 100

print(f"Trenutni prag (G1 < 10):")
print(f"  - Učenici u riziku: {risk_count} ({risk_pct:.2f}%)")
print(f"  - Učenici koji prolaze: {pass_count} ({100-risk_pct:.2f}%)")

# Testiranje alternativnih pragova
print(f"\n{'='*60}")
print("\nANALIZA ALTERNATIVNIH PRAGOVA:")
print("-" * 50)

thresholds = [8, 9, 10, 11, 12, 13]
for threshold in thresholds:
    risk = (df['G1'] < threshold).sum()
    risk_pct = risk / len(df) * 100
    status = ""
    if 15 <= risk_pct <= 50:
        status = "✓ OPTIMALNO"
    elif 10 <= risk_pct < 15 or 50 < risk_pct <= 60:
        status = "⚠️ PRIHVATLJIVO"
    else:
        status = "❌ NEBALANSIRANO"
    print(f"  Prag G1 < {threshold}: {risk:3d} učenika u riziku ({risk_pct:5.2f}%) {status}")

print(f"\n{'='*60}")

# Finalna preporuka
print("\nKONAČNA PREPORUKA:")
if 10 <= risk_pct <= 50:
    print(f"  ✓ Trenutni prag (G1 < 10) je PRIHVATLJIV za modeliranje.")
    print(f"    Manjinska klasa (rizik) ima {risk_pct:.2f}% zastupljenosti.")
    print("    Nije potreban redizajn ciljne varijable.")
elif risk_count == 0:
    print("  ❌ KRITIČNO: Nema učenika s ocjenom < 10!")
    print("    POTREBAN REDIZAJN - predlaže se viši prag (npr. G1 < 12).")
else:
    print(f"  ⚠️ Razmotrite prilagodbu praga za bolju balansiranost klasa.")


ANALIZA POTREBE ZA REDIZAJNOM CILJNE VARIJABLE:

Trenutni prag (G1 < 10):
  - Učenici u riziku: 157 (24.19%)
  - Učenici koji prolaze: 492 (75.81%)


ANALIZA ALTERNATIVNIH PRAGOVA:
--------------------------------------------------
  Prag G1 < 8:  50 učenika u riziku ( 7.70%) NEBALANSIRANO
  Prag G1 < 9:  92 učenika u riziku (14.18%) PRIHVATLJIVO
  Prag G1 < 10: 157 učenika u riziku (24.19%) OPTIMALNO
  Prag G1 < 11: 252 učenika u riziku (38.83%) OPTIMALNO
  Prag G1 < 12: 343 učenika u riziku (52.85%) PRIHVATLJIVO
  Prag G1 < 13: 425 učenika u riziku (65.49%) NEBALANSIRANO


KONAČNA PREPORUKA:
Razmotrite prilagodbu praga za bolju balansiranost klasa.


### 5.5 Sažetak - Validacija Ciljne Varijable

Na temelju provedene analize, ovdje sumiramo status ciljne varijable:


In [22]:
# Finalni sažetak validacije ciljne varijable

print("=" * 60)
print("SAŽETAK VALIDACIJE CILJNE VARIJABLE")
print("=" * 60)

print(f"\n1. DEFINICIJA:")
print(f"   Naziv: target_G1")
print(f"   Tip: Binomna (0/1)")
print(f"   Pravilo: 1 ako G1 < 10 (rizik), 0 ako G1 >= 10 (prolaz)")

print(f"\n2. DISTRIBUCIJA:")
print(f"   Prolaz (0): {freq[0]} učenika ({freq_pct[0]:.2f}%)")
print(f"   Rizik  (1): {freq[1]} učenika ({freq_pct[1]:.2f}%)")
print(f"   Omjer: {ratio:.2f}:1")

print(f"\n3. VREMENSKO USKLAĐIVANJE:")
print(f"   ✓ Svi prediktori vremenski prethode G1")
print(f"   ✓ G2 i G3 isključeni iz prediktora (nastaju nakon G1)")

print(f"\n4. KVALITETA ZA MODELIRANJE:")
if min_class_pct >= 15:
    status = "ODLIČNA"
elif min_class_pct >= 10:
    status = "PRIHVATLJIVA"
else:
    status = "POTREBNO BALANSIRANJE"
print(f"   Status: {status}")

print(f"\n{'='*60}")
print("\nCiljna varijabla 'target_G1' je definirana, validirana i spremna")
print("za korištenje u daljnjoj analizi relevantnosti atributa.")
print("=" * 60)


SAŽETAK VALIDACIJE CILJNE VARIJABLE

1. DEFINICIJA:
   Naziv: target_G1
   Tip: Binomna (0/1)
   Pravilo: 1 ako G1 < 10 (rizik), 0 ako G1 >= 10 (prolaz)

2. DISTRIBUCIJA:
   Prolaz (0): 492 učenika (75.81%)
   Rizik  (1): 157 učenika (24.19%)
   Omjer: 3.13:1

3. VREMENSKO USKLAĐIVANJE:
   Svi prediktori vremenski prethode G1
   G2 i G3 isključeni iz prediktora (nastaju nakon G1)

4. KVALITETA ZA MODELIRANJE:
   Status: ODLIČNA


Ciljna varijabla 'target_G1' je definirana, validirana i spremna
za korištenje u daljnjoj analizi relevantnosti atributa.
